# Implicit Parameters

### Review: Function Calling Syntax

In [6]:
def f(x: Int, y: Int) = x + y

defined function f

In [7]:
def g(x: Int)(y: Int) = x + y

defined function g

In [8]:
g(1)(3)

res7: Int = 4

In [12]:
f(3, 1)

res11: Int = 4

In [14]:
def process(i: Int)(f: Int => Int) = f(i * 2)

process(10) {  
    (i: Int) => i + 1 
}

defined function process
res13_1: Int = 21

# Implicit Parameters

In [1]:
implicit val anyName: String = "LOG: "

anyName: String = "LOG: "

In [2]:
def show(msg: String)(implicit p: String) = println(p + msg)

defined function show

In [15]:
show("Hi")("ERROR: ")

ERROR: Hi


In [16]:
show("Hi")

LOG: Hi


In [17]:
show("Hi")(anyName)

LOG: Hi


## Use of Implicit Parameters

* implicit parameter:
    * general api depedency/configuration
    * query() has a "context" of running
    * "context" is the connection

In [33]:
def queryTable(sql: String)(conn: (String, Int)) = 
    println(sql, conn)

queryTable("SELECT * FROM users")(("UK", 123))

(SELECT * FROM users,(UK,123))


defined function queryTable

In [34]:
def queryTable(sql: String)(conn: (String, Int)) = 
    println(sql, conn)

val conn = ("UK", 123)

queryTable("SELECT * FROM users")(conn)

(SELECT * FROM users,(UK,123))


defined function queryTable
conn: (String, Int) = ("UK", 123)

In [35]:
def queryTable(sql: String)(implicit conn: (String, Int)) = 
    println(sql, conn)

implicit val conn = ("UK", 123)
queryTable("SELECT * FROM users")(conn)

(SELECT * FROM users,(UK,123))


defined function queryTable
conn: (String, Int) = ("UK", 123)

In [37]:
def queryTable(sql: String)(implicit conn: (String, Int)) = 
    println(sql, conn)

implicit val conn: (String, Int) = ("UK", 123)

queryTable("SELECT * FROM users")

(SELECT * FROM users,(UK,123))


defined function queryTable
conn: (String, Int) = ("UK", 123)

In [22]:
implicit val conn: (String, Int) = ("UK", 1234)

def query(sql: String)(implicit h: (String, Int)) = {
    println(s"quering $sql @$h")
}

conn: (String, Int) = ("UK", 1234)
defined function query

In [23]:
query("SELCT * FROM users")
query("SELCT * FROM people")
query("SELCT * FROM forums")
query("SELCT * FROM blogs")

quering SELCT * FROM users @(UK,1234)
quering SELCT * FROM people @(UK,1234)
quering SELCT * FROM forums @(UK,1234)
quering SELCT * FROM blogs @(UK,1234)


## Exercise: Implicit Parameters

* define:
    * select()
    * insert()
    * delete()
    
* that take a sql: String, and an implicit dbname : String
    * each should simulate the relevant query, println()
    
* run the following queries:
    ```
    "SELECT * FROM users"
    "INSERT INTO users (1, 2)"
    "DElETE FROM users"
    ```
* and show they all run on the same db

In [1]:
implicit val db: String = "ukmk"

def select(sql: String)(implicit dbname: String) = 
    println("selecting", sql, dbname)

def insert(sql: String)(implicit dbname: String) = 
    println("inserting", sql, dbname)

def delete(sql: String)(implicit dbname: String) = 
    println("deleting", sql, dbname)


select("SELECT * FROM users")
insert("INSERT INTO users")
delete("DELETE FROM users")

(selecting,SELECT * FROM users,ukmk)
(inserting,INSERT INTO users,ukmk)
(deleting,DELETE FROM users,ukmk)


db: String = "ukmk"
defined function select
defined function insert
defined function delete

## Exercise


In [23]:
/* class, case class, trait, abstract class */
trait DbConfig {
    val host: String
    val pw: Int
}

/* class Config extends DbConfig; conf = new Config */

implicit object conf extends DbConfig {
    val host = "UK"
    val pw = 1234
}


def select(sql: String)(implicit d: DbConfig) = 
    println("selecting", sql, d.host)

def insert(sql: String)(implicit d: DbConfig) = 
    println("inserting", sql, d.host)

def delete(sql: String)(implicit d: DbConfig) = 
    println("deleting", sql, d.host)


select("SELECT * FROM users")
insert("INSERT INTO users")
delete("DELETE FROM users")

(selecting,SELECT * FROM users,UK)
(inserting,INSERT INTO users,UK)
(deleting,DELETE FROM users,UK)


defined trait DbConfig
defined object conf
defined function select
defined function insert
defined function delete

## Exercise

* modify select(), delete(), insert() to take an `implicit` DbConfig
* DbConfig has a:
    - db name
    - usr name
    - pwd
* provide an implicit object with UK-based configuration

* HINT: see example below

In [4]:
trait DbName {
    val name: String
}

implicit object dbname extends DbName {
    val name = "UK"
}

def query(s: String)(implicit n: DbName) = println(s + n.name)
query("sxf")

sxfUK


defined trait DbName
defined object dbname
defined function query

In [30]:
//implicit val value: String = "Michael"


trait Data[A] {
    val value: String
}

trait User

// implicit val UkHost = new ...generatedclass ...
implicit object UkUser extends Data[User] {
    val value = "Michael"
}


trait Host
// implicit val UkHost...
implicit object UkHost extends Data[Host] {
    val value = "UK"
}



defined trait Data
defined trait User
defined object UkUser
defined trait Host
defined object UkHost

In [27]:
def select(s: String)(implicit h: Data[Host], u: Data[User]) = 
    println(s, h.value, u.value)

defined function select

In [28]:
select("Hi")

(Hi,UK,Michael)


## Multiple Implicits & Scoping

* application vs. framework
* solving the problems vs. solving the problems of applicaiton developers



* app developer, client = biz
* framework developer, client = application
* technical architect, client = framework developer
* external libr. designer, client = TA/FD/AD
* language design, client = FW | APP | EDU

In [1]:
// framework developer provides `query` and a type to configure `query`
trait Config {
    val name: String
}

def query(s: String)(implicit n: Config) = println(s + n.name)

defined trait Config
defined function query

In [2]:
// lead application developer
object Configs {
    implicit object confUK extends Config {
        val name = "UK"
    }

    implicit object confFR extends Config {
        val name = "FR"
    }
}

defined object Configs

In [4]:
//app developer
import Configs.confUK
query("SQL ")

SQL UK


import Configs.confUK


## Exercise: Select an Import

* Define multiple DbConfigs, wrap in ConfigContext object
* Import one of them